<a href="https://colab.research.google.com/github/catarinacecilio/learning_imagery/blob/master/learning_imagery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
from PIL import Image
import sys
import csv

# Raw Data

In [ ]:
!tar -xvzf /content/NEKOa.tgz -C /content/raw_data
!tar -xvzf /content/NEKOb.tgz -C /content/raw_data
!tar -xvzf /content/NEKOc.tgz -C /content/raw_data
!tar -xvzf /content/PETEc.tgz -C /content/raw_data
!tar -xvzf /content/CompleteAnnotations_2016-07-11.tgz -C /content/raw_data

In [ ]:
roi_df = pd.read_csv('raw_data/roi.tsv', sep='\,')
camera_names = roi_df['Camera name']
camera_names

In [ ]:
# default format can be changed as needed
def createFileList(myDir, format='.jpg'):
    fileList = []
    print(myDir)
    labels = []
    names = []
    keywords = {"K" : "1","U": "0",} # keys and values to be changed as needed
    for root, dirs, files in os.walk(myDir, topdown=True):
        for name in files:
            if name.endswith(format):
                fullName = os.path.join(root, name)
                fileList.append(fullName)
            for keyword in keywords:
                if keyword in name:
                    labels.append(keywords[keyword])
                else:
                    continue
            names.append(name)
    return fileList, labels, names

In [ ]:
# load the original image
myFileList, labels, names  = createFileList('/content/raw_data/')
n_files = len(names)

In [ ]:
i = 0
for file in myFileList:
    print(file)
    img_file = Image.open(file)
    # img_file.show()
# get original image parameters...
    width, height = img_file.size
    format = img_file.format
    mode = img_file.mode
# Make image Greyscale
    img_grey = img_file.convert('L')
    #img_grey.save('result.png')
    #img_grey.show()
# Save Greyscale values
    value = np.asarray(img_grey.getdata(), dtype=np.int).reshape((width, height))
    value = value.flatten()
    
    value = np.append(value,labels[i])
    i +=1
    
    print(value)
    with open("penguim_dataset.csv", 'a') as f:
        writer = csv.writer(f)
        writer.writerow(value)